In [ ]:
# ===============================================================
#         Segmentation comportementale des clients
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# Chargement du dataset nettoyé
df = pd.read_csv("Camp_Market_cleaned.csv", sep=";")

# --- Sélection des variables comportementales ---
features_behavior = [
    "Recency",
    "NumWebPurchases",
    "NumStorePurchases",
    "NumCatalogPurchases",
    "NumWebVisitsMonth",
    "TotalSpent",
    "AcceptedCmp_total"
]

# Ajout du ratio d'achat web
if "WebPurchaseRatio" not in df.columns:
    df["WebPurchaseRatio"] = df["NumWebPurchases"] / (df["NumWebPurchases"] + df["NumStorePurchases"] + 1)
    features_behavior.append("WebPurchaseRatio")

# Remplissage des valeurs manquantes
df[features_behavior] = df[features_behavior].fillna(0)

# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features_behavior])

# Méthode du coude pour déterminer le bon nombre de clusters
inertia = []
for k in range(2,7):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X_scaled)
    inertia.append(model.inertia_)

plt.plot(range(2,7), inertia, marker='o')
plt.title("Méthode du coude (comportement)")
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie")
plt.show()

# K-Means final (ex: 4 clusters)
kmeans = KMeans(n_clusters=4, random_state=42)
df["Cluster_Behavior"] = kmeans.fit_predict(X_scaled)

# Visualisation
display_cols = ["TotalSpent", "Recency", "NumWebPurchases", "WebPurchaseRatio"]
sns.pairplot(df, hue="Cluster_Behavior", vars=display_cols)
plt.suptitle("Clustering comportemental", y=1.02)
plt.show()

# Résumé des moyennes par cluster
cluster_summary = df.groupby("Cluster_Behavior")[features_behavior].mean()
print("\n📊 Moyennes comportementales par cluster :")
print(cluster_summary)

# Sauvegarde
df.to_csv("Camp_Market_Comportement.csv", index=False)
cluster_summary.to_csv("Comportement_Clusters.csv")

print("\n✅ Fichiers exportés : Camp_Market_Comportement.csv et Comportement_Clusters.csv")
